# My plan so far after experimenting with the data:

drop all cols except for make, model, eventid, event date, amateur built, weather condition, engine type, and the injury totals.

Clean the data using as much of the code I have already written as possible. Define functions early for operations I think will be commmon so I don't have to keep copy pasting them.

group by both make and model in the main df. Make separate DFs that all additionally aggregate based on injury totals, as well as one of amateur built, weather condition, and engine type.

create a statistic that represents the liklihood of any one make an model to be in an accident vs another. Then, add in the comparison of whether they are amateur built, the weather conditions at the time of the incident, and the engine type to see how it changes the distribution.

recommend whether or not amateur built aircraft are safe compared to others

recommend specific makes and models that perform well in bad weather conditions and single out those that do not perform

recommend planes that have a specific engine type that performs well?

make a recommendation of what the best overall aircraft is based on all of these analyses.

## Step 1: import packages and read the CSV into memory

In [1]:
### import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
### load csv into dataframe
df = pd.read_csv('data/Aviation_Data.csv', low_memory=False)
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Step 2: remove as many uncessary columns as possible

In [2]:
### find total number of NaN entries in each column
df.isna().sum()

Event.Id                   1459
Investigation.Type            0
Accident.Number            1459
Event.Date                 1459
Location                   1511
Country                    1685
Latitude                  55966
Longitude                 55975
Airport.Code              40099
Airport.Name              37558
Injury.Severity            2459
Aircraft.damage            4653
Aircraft.Category         58061
Registration.Number        2776
Make                       1522
Model                      1551
Amateur.Built              1561
Number.of.Engines          7543
Engine.Type                8536
FAR.Description           58325
Schedule                  77766
Purpose.of.flight          7651
Air.carrier               73700
Total.Fatal.Injuries      12860
Total.Serious.Injuries    13969
Total.Minor.Injuries      13392
Total.Uninjured            7371
Weather.Condition          5951
Broad.phase.of.flight     28624
Report.Status              7840
Publication.Date          16689
dtype: i

In [3]:
### find total number of duplicate rows
df.duplicated().value_counts()

False    88958
True      1390
dtype: int64

In [4]:
### drop all duplicate rows and confirm there are no more duplicates
df = df.drop_duplicates()
df.duplicated().value_counts()

False    88958
dtype: int64

In [5]:
### focusing in on event.id column. It seems like the NaN entries in this column are lacking all data other than the
### investigation type, so there isn't really a reason to keep any of them.
df[df['Event.Id'].isna()]

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
64030,NaN,25-09-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69843,NaN,14-06-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74420,NaN,03-11-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74946,NaN,24-02-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75894,NaN,02-02-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89440,NaN,02-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89492,NaN,05-08-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89763,NaN,08-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89943,NaN,15-12-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df=df.dropna(subset=['Event.Id'])
df[df['Event.Id'].isna()]

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date


In [7]:
### confirming we got rid of the event.id NaN entries. It looks like the NaN entries in accdient.number and event.date just
### happened to be cleared out by the last operation as well.
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date          15230
dtype: i

In [8]:
### location seems to be a categorical variable with less than 1% of values as NaN. In this case, I think it makes a lot more 
### sense to just remove these NaN values.
df['Location'].value_counts()

ANCHORAGE, AK                      434
MIAMI, FL                          200
ALBUQUERQUE, NM                    196
HOUSTON, TX                        193
CHICAGO, IL                        184
                                  ... 
Three Points, AZ                     1
Commune de Brethencourt, France      1
Gunthertown, AL                      1
STANFORD, CA                         1
ST REGIS, MT                         1
Name: Location, Length: 27758, dtype: int64

In [9]:
df=df.dropna(subset=['Location'])
df['Location'].isna().sum()

0

In [10]:
### time to check how that last operation affected the overall data!
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                     225
Latitude                  54460
Longitude                 54469
Airport.Code              38590
Airport.Name              36049
Injury.Severity            1000
Aircraft.damage            3187
Aircraft.Category         56559
Registration.Number        1295
Make                         63
Model                        92
Amateur.Built                98
Number.of.Engines          6055
Engine.Type                7071
FAR.Description           56823
Schedule                  76288
Purpose.of.flight          6182
Air.carrier               72192
Total.Fatal.Injuries      11387
Total.Serious.Injuries    12492
Total.Minor.Injuries      11914
Total.Uninjured            5901
Weather.Condition          4487
Broad.phase.of.flight     27127
Report.Status              6381
Publication.Date          15218
dtype: i

In [11]:
df['Country'].value_counts().head(20)

United States     82237
Brazil              373
Canada              359
Mexico              358
United Kingdom      344
Australia           298
France              233
Spain               226
Bahamas             216
Germany             215
Colombia            193
South Africa        129
Japan               126
Venezuela           121
Italy               114
Argentina           111
Indonesia           109
India                96
Peru                 93
Russia               91
Name: Country, dtype: int64

In [12]:
### the country column contains categorical data and is only missing less than 1% of values. It makes the most sense to just
### drop the values here again.
df=df.dropna(subset=['Country'])
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  54236
Longitude                 54245
Airport.Code              38432
Airport.Name              35903
Injury.Severity            1000
Aircraft.damage            3171
Aircraft.Category         56346
Registration.Number        1290
Make                         62
Model                        91
Amateur.Built                98
Number.of.Engines          6048
Engine.Type                7065
FAR.Description           56610
Schedule                  76156
Purpose.of.flight          6175
Air.carrier               72022
Total.Fatal.Injuries      11376
Total.Serious.Injuries    12482
Total.Minor.Injuries      11904
Total.Uninjured            5892
Weather.Condition          4482
Broad.phase.of.flight     27117
Report.Status              6381
Publication.Date          15161
dtype: i

In [13]:
### The next four columns contain massive amounts of missing entries. However, I want to use the information they contain in 
### part of my analysis later. I am going to skip over them for now, and once I have cleaned the rest of the data, I will
### create two dataframes, one which has the coordinates and airport data with the missing entries removed, and one in which 
### these columns have been dropped. Instead, I will move on to the injury.severity column.

df['Injury.Severity'].value_counts().head(20)

Non-Fatal      67195
Fatal(1)        6137
Fatal           5262
Fatal(2)        3689
Incident        2187
Fatal(3)        1136
Fatal(4)         806
Fatal(5)         231
Minor            218
Serious          173
Fatal(6)         161
Unavailable       92
Fatal(7)          55
Fatal(8)          51
Fatal(10)         31
Fatal(9)          17
Fatal(14)         11
Fatal(11)         10
Fatal(13)          9
Fatal(12)          8
Name: Injury.Severity, dtype: int64

In [14]:
### I've noticed there seem to be several different buckets in this column that are extraneous for the purpose of this 
### analysis. It does not seem like it would be worth differentiating between the severities of different types of non-fatal 
### injury, so I am going to lump them all in with the non-fatal category. While I'm doing this, I will drop the rows that 
### contain "unavailable", since they are a small subset of total values and are a placeholder value.

df['Injury.Severity'] = df['Injury.Severity'].replace(to_replace=['Incident', 'Minor', 'Serious'], value='Non-Fatal')
df = df.drop(df[df['Injury.Severity'] == 'Unavailable'].index)
df.dropna(axis=0, subset=['Injury.Severity'], inplace=True)
df['Injury.Severity'].value_counts().head(20)

Non-Fatal    69773
Fatal(1)      6137
Fatal         5262
Fatal(2)      3689
Fatal(3)      1136
Fatal(4)       806
Fatal(5)       231
Fatal(6)       161
Fatal(7)        55
Fatal(8)        51
Fatal(10)       31
Fatal(9)        17
Fatal(14)       11
Fatal(11)       10
Fatal(13)        9
Fatal(12)        8
Fatal(18)        5
Fatal(20)        5
Fatal(23)        4
Fatal(25)        4
Name: Injury.Severity, dtype: int64

In [15]:
### The existence of numbered fatal entries in this column is redundant because of the other columns dedicated to this statistic.
### I think it makes sense to turn this column into a simple dichotomy between fatal and non-fatal.

df['Injury.Severity'] = df['Injury.Severity'].map(lambda x: 'Fatal' if x[:5]=='Fatal' else x)

In [16]:
df['Injury.Severity'].value_counts().head(20)

Non-Fatal    69773
Fatal        17747
Name: Injury.Severity, dtype: int64

In [17]:
df['Aircraft.damage'].value_counts().head(4)

Substantial    63779
Destroyed      18430
Minor           2560
Unknown           92
Name: Aircraft.damage, dtype: int64

In [18]:
### This column seems to be lacking any special cases, and I think I can just deal with it the way I dealt with the previous
### column since we are still at less than 5% of total values as placeholders or NaN. I will drop the NaN values and remove
### the rows with 'Unknown'.
df = df.drop(df[df['Aircraft.damage'] == 'Unknown'].index)
df.dropna(axis=0, subset=['Aircraft.damage'], inplace=True)
df['Aircraft.damage'].value_counts().head(20)

Substantial    63779
Destroyed      18430
Minor           2560
Name: Aircraft.damage, dtype: int64

In [19]:
### Time for the next column on the list. Let's take a look at the aircraft category and carefully consider what to do with it,
### since it seems to be missing values for half of all entries.
df['Aircraft.Category'].value_counts().head(20)

Airplane             25830
Helicopter            3292
Glider                 503
Gyrocraft              173
Weight-Shift           160
Balloon                135
Powered Parachute       88
Ultralight              29
Unknown                 11
WSFT                     9
Powered-Lift             4
Blimp                    4
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64

In [20]:
### Just looking at the values that are contained within that column, I've made a realization. Is the majority of the data in
### my dataset not even related to actual airplanes? I need to take some time to really consider if it is worth keeping ANY
### of the rows where I can't confirm that the individual data points I'm working with are not airplanes, since they are not
### at all relevant to the business analysis I have been tasked with doing.

### For now, I have decided to just drop this column since I think I can get the information I need just from the make and model
### columns. I will revisit it later if I can figure out a way to use this data.

df=df.drop(axis=1, labels='Aircraft.Category')

In [21]:
df['Make'].value_counts().head(20)

Cessna         22021
Piper          11916
CESSNA          4719
Beech           4265
PIPER           2756
Bell            2012
Mooney          1084
Grumman         1083
Boeing          1013
BEECH           1001
Robinson         933
Bellanca         882
Hughes           780
Schweizer        621
Air Tractor      593
BELL             547
Aeronca          483
Maule            443
Champion         426
BOEING           390
Name: Make, dtype: int64

In [22]:
df.dropna(axis=0, subset=['Make'], inplace=True)

In [23]:
df['Model'].value_counts().head(20)

152          2346
172          1717
172N         1147
PA-28-140     925
150           817
172M          791
172P          684
182           650
180           620
150M          583
PA-18         577
PA-18-150     576
PA-28-180     569
PA-28-161     564
PA-28-181     524
206B          500
PA-38-112     464
G-164A        459
150L          454
A36           443
Name: Model, dtype: int64

In [24]:
df.dropna(axis=0, subset=['Model'], inplace=True)

In [25]:
### Considering the above two columns, I believe that the aircraft category column does not contain very good information, and 
### I think it would be absurd for only a minority of these data points to represent planes, considering the 'make' data entries.

In [26]:
df['Registration.Number'].value_counts().head(20)

NONE      333
UNREG     121
None       64
N20752      7
N8402K      6
N121CC      6
N11VH       6
N53893      6
N4101E      6
UNK         6
N5408Y      6
N8653Y      5
N3331R      5
N93067      5
N5246E      5
N99HV       5
N75LE       5
N9957J      5
N3125N      5
N8597D      5
Name: Registration.Number, dtype: int64

In [27]:
### Now this is interesting. Aircraft registration numbers are supposed to be unique, but it seems like many of these numbers
### have been involved in numerous incidents. There are also three different bins for reports with no registration numbers
### when there really only needs to be one.

In [28]:
df['Registration.Number'] = df['Registration.Number'].replace(to_replace=['UNREG', 'None'], value='NONE')
df.dropna(subset=['Registration.Number'], inplace=True)
df['Registration.Number'].value_counts().head(20)

NONE      518
N20752      7
N5408Y      6
N121CC      6
UNK         6
N8402K      6
N11VH       6
N4101E      6
N53893      6
N93067      5
N3125N      5
N9957J      5
N420SB      5
N3331R      5
N8597D      5
N99HV       5
N5246E      5
N8653Y      5
N75LE       5
N4563B      4
Name: Registration.Number, dtype: int64

In [29]:
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  50809
Longitude                 50819
Airport.Code              34803
Airport.Name              32318
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                26
Number.of.Engines          3445
Engine.Type                4572
FAR.Description           53899
Schedule                  73896
Purpose.of.flight          3628
Air.carrier               68905
Total.Fatal.Injuries      10481
Total.Serious.Injuries    11421
Total.Minor.Injuries      10649
Total.Uninjured            5107
Weather.Condition          2536
Broad.phase.of.flight     23664
Report.Status              4774
Publication.Date          14197
dtype: int64

In [30]:
df['Amateur.Built'].value_counts().head()

No     75277
Yes     8365
Name: Amateur.Built, dtype: int64

In [31]:
### I think it makes more sense for this column to be boolean type data, since there are only two options. While I'm at it, 
### I'll just drop the NaNs.
df.dropna(subset=['Amateur.Built'], inplace=True)
df['Amateur.Built'] = df['Amateur.Built'].map(lambda x: True if x=='Yes' else False)
df['Amateur.Built'].value_counts().head()

False    75277
True      8365
Name: Amateur.Built, dtype: int64

In [32]:
df['Number.of.Engines'].value_counts().head(10)

1.0    68889
2.0     9655
0.0     1046
3.0      317
4.0      297
8.0        2
6.0        1
Name: Number.of.Engines, dtype: int64

In [33]:
### for the number of engines, I am also just going to drop the NaNs because it is still below 5% of the data. This is a 
### case where I think it wouldn't be a bad idea to just fill in the missing entries with 1.0, but I want to maintain the
### consistency of my technique, and I don't think I'm losing too much accuracy here by dropping.

df.dropna(subset=['Number.of.Engines'], inplace=True)
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  48966
Longitude                 48976
Airport.Code              32416
Airport.Name              29952
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                2634
FAR.Description           52540
Schedule                  70886
Purpose.of.flight          2334
Air.carrier               66384
Total.Fatal.Injuries      10238
Total.Serious.Injuries    11126
Total.Minor.Injuries      10356
Total.Uninjured            4950
Weather.Condition          1141
Broad.phase.of.flight     20965
Report.Status              3125
Publication.Date          13674
dtype: int64

In [34]:
df['Engine.Type'].value_counts().head(10)

Reciprocating      68484
Turbo Shaft         3352
Turbo Prop          3065
Turbo Fan           1481
Unknown              640
Turbo Jet            525
None                  13
Electric               7
NONE                   2
Geared Turbofan        1
Name: Engine.Type, dtype: int64

In [35]:
### Since this column already contains an unknown bin, I will fill the NaNs with unknown while I standardize the data.

df['Engine.Type'].fillna(value='Unknown', inplace=True)
df['Engine.Type'] = df['Engine.Type'].replace(to_replace=['NONE', 'None'], value='Unknown')
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  48966
Longitude                 48976
Airport.Code              32416
Airport.Name              29952
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                   0
FAR.Description           52540
Schedule                  70886
Purpose.of.flight          2334
Air.carrier               66384
Total.Fatal.Injuries      10238
Total.Serious.Injuries    11126
Total.Minor.Injuries      10356
Total.Uninjured            4950
Weather.Condition          1141
Broad.phase.of.flight     20965
Report.Status              3125
Publication.Date          13674
dtype: int64

In [36]:
df['FAR.Description'].value_counts().head(10)

091                              17294
Part 91: General Aviation         6348
137                                949
135                                680
NUSN                               574
Part 137: Agricultural             433
Part 135: Air Taxi & Commuter      284
121                                259
PUBU                               235
NUSC                               187
Name: FAR.Description, dtype: int64

In [37]:
### I'm skipping that column for now because I don't know what it represents or if it will be useful to me.

df['Schedule'].value_counts().head(10)

UNK     3822
NSCH    3670
SCHD    1829
Name: Schedule, dtype: int64

In [38]:
### These values stand for unkown, not scheduled, and scheduled. While I think this data would be useful to my analysis, we 
### only have values in it for a minority of the rows. I'm just going to drop this column.

df=df.drop(axis=1, labels='Schedule')

In [39]:
df['Purpose.of.flight'].value_counts().head(20)

Personal                     47547
Instructional                10235
Unknown                       5261
Aerial Application            4582
Business                      3772
Positioning                   1530
Other Work Use                1136
Ferry                          766
Aerial Observation             709
Public Aircraft                648
Executive/corporate            491
Flight Test                    372
Skydiving                      173
External Load                  104
Banner Tow                      99
Public Aircraft - Federal       94
Air Race show                   73
Public Aircraft - Local         69
Public Aircraft - State         59
Air Race/show                   54
Name: Purpose.of.flight, dtype: int64

In [40]:
### This column is useful to me and contains an unknown bin, so I will fill the NaNs will Unknown.

df['Purpose.of.flight'].fillna(inplace=True, value='Unknown')
df['Purpose.of.flight'].value_counts().head(20)

Personal                     47547
Instructional                10235
Unknown                       7595
Aerial Application            4582
Business                      3772
Positioning                   1530
Other Work Use                1136
Ferry                          766
Aerial Observation             709
Public Aircraft                648
Executive/corporate            491
Flight Test                    372
Skydiving                      173
External Load                  104
Banner Tow                      99
Public Aircraft - Federal       94
Air Race show                   73
Public Aircraft - Local         69
Public Aircraft - State         59
Air Race/show                   54
Name: Purpose.of.flight, dtype: int64

In [41]:
### The air carrier column is not useful to my analysis, so I am going to drop it from the data.

df=df.drop(axis=1, labels='Air.carrier')
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  48966
Longitude                 48976
Airport.Code              32416
Airport.Name              29952
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                   0
FAR.Description           52540
Purpose.of.flight             0
Total.Fatal.Injuries      10238
Total.Serious.Injuries    11126
Total.Minor.Injuries      10356
Total.Uninjured            4950
Weather.Condition          1141
Broad.phase.of.flight     20965
Report.Status              3125
Publication.Date          13674
dtype: int64

In [42]:
df['Total.Fatal.Injuries'].value_counts()

0.0      54507
1.0       7910
2.0       4615
3.0       1362
4.0        918
         ...  
256.0        1
15.0         1
217.0        1
92.0         1
156.0        1
Name: Total.Fatal.Injuries, Length: 66, dtype: int64

In [43]:
df['Total.Serious.Injuries'].value_counts()

0.0      57678
1.0       7957
2.0       2567
3.0        538
4.0        204
5.0         53
6.0         21
7.0         17
8.0          7
9.0          7
13.0         4
10.0         4
11.0         2
28.0         2
26.0         2
47.0         1
125.0        1
25.0         1
81.0         1
41.0         1
67.0         1
17.0         1
20.0         1
45.0         1
137.0        1
14.0         1
60.0         1
15.0         1
19.0         1
12.0         1
16.0         1
27.0         1
53.0         1
Name: Total.Serious.Injuries, dtype: int64

In [44]:
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  48966
Longitude                 48976
Airport.Code              32416
Airport.Name              29952
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                   0
FAR.Description           52540
Purpose.of.flight             0
Total.Fatal.Injuries      10238
Total.Serious.Injuries    11126
Total.Minor.Injuries      10356
Total.Uninjured            4950
Weather.Condition          1141
Broad.phase.of.flight     20965
Report.Status              3125
Publication.Date          13674
dtype: int64

In [45]:
df.dropna(subset=['Weather.Condition'], inplace=True)

In [46]:
df.dropna(subset=['Report.Status'], inplace=True)

In [47]:
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  48531
Longitude                 48541
Airport.Code              30909
Airport.Name              28292
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                   0
FAR.Description           52401
Purpose.of.flight             0
Total.Fatal.Injuries      10219
Total.Serious.Injuries    11090
Total.Minor.Injuries      10319
Total.Uninjured            4924
Weather.Condition             0
Broad.phase.of.flight     17735
Report.Status                 0
Publication.Date          13319
dtype: int64

In [48]:
df=df.drop(axis=1, labels=['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'FAR.Description', 'Publication.Date', 'Broad.phase.of.flight'])

In [49]:
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Injury.Severity               0
Aircraft.damage               0
Registration.Number           0
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines             0
Engine.Type                   0
Purpose.of.flight             0
Total.Fatal.Injuries      10219
Total.Serious.Injuries    11090
Total.Minor.Injuries      10319
Total.Uninjured            4924
Weather.Condition             0
Report.Status                 0
dtype: int64